In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow import keras
from sklearn.model_selection import train_test_split
# ---------------------------
# CARGAR LOS MODELOS (.keras)
# ---------------------------
model_BR = keras.models.load_model('modelo_reentrenado_keras_BR.keras')
model_STB = keras.models.load_model('modelo_reentrenado_keras_STB.keras')


2025-04-27 18:01:49.882742: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-27 18:01:49.886595: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-27 18:01:49.898468: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745787709.919106  106460 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745787709.924765  106460 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745787709.939732  106460 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [7]:
# ---------------------------
# CARGAR NUEVOS DATOS
# ---------------------------
df = pd.read_excel('NUAR-B_FEBRERO2025_CONVERTIO_1.xlsx', parse_dates=["DATE/TIME(America/Asuncion)"])

# Convertir fecha a datetime
df['DATE/TIME(America/Asuncion)'] = pd.to_datetime(df['DATE/TIME(America/Asuncion)'], errors='coerce')

# Extraer componentes de fecha y hora
df['dia'] = df['DATE/TIME(America/Asuncion)'].dt.day
df['mes'] = df['DATE/TIME(America/Asuncion)'].dt.month
df['año'] = df['DATE/TIME(America/Asuncion)'].dt.year
df['hora'] = df['DATE/TIME(America/Asuncion)'].dt.hour
df['minuto'] = df['DATE/TIME(America/Asuncion)'].dt.minute
df['segundo'] = df['DATE/TIME(America/Asuncion)'].dt.second

# Eliminar columna de fecha
df = df.drop('DATE/TIME(America/Asuncion)', axis=1)

In [8]:
# Separar entradas (X) y salidas (y)
X = df.drop(columns=["Flow Rate Prop. BR", "Flow Rate Prop. STB"])
y = df[["Flow Rate Prop. BR", "Flow Rate Prop. STB"]]

# Imputar valores nulos con la mediana
imputer = SimpleImputer(strategy="median")
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
y_imputed = pd.DataFrame(imputer.fit_transform(y), columns=y.columns)

# Dividir en entrenamiento y prueba (mismo índice para ambos targets)
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_imputed, test_size=0.1, random_state=24)

y_train_BR = y_train["Flow Rate Prop. BR"]
y_test_BR = y_test["Flow Rate Prop. BR"]
y_train_STB = y_train["Flow Rate Prop. STB"]
y_test_STB = y_test["Flow Rate Prop. STB"]



In [9]:
# Escalar datos como en el entrenamiento
scaler_X = StandardScaler()
scaler_X.fit(X_train)  # Entrenar el scaler con los datos de training
X_train_scaled = scaler_X.transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Escalado de Y (los targets)
scaler_y_BR = StandardScaler()
scaler_y_STB = StandardScaler()

y_train_BR_scaled = scaler_y_BR.fit_transform(y_train_BR.values.reshape(-1, 1))
y_test_BR_scaled = scaler_y_BR.transform(y_test_BR.values.reshape(-1, 1))

y_train_STB_scaled = scaler_y_STB.fit_transform(y_train_STB.values.reshape(-1, 1))
y_test_STB_scaled = scaler_y_STB.transform(y_test_STB.values.reshape(-1, 1))


# Predecir
y_pred_BR_scaled = model_BR.predict(X_test_scaled)
y_pred_STB_scaled = model_STB.predict(X_test_scaled)



12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [10]:
# Invertir escala
y_pred_BR_inv = scaler_y_BR.inverse_transform(y_pred_BR_scaled)
y_test_BR_inv = scaler_y_BR.inverse_transform(y_test_BR_scaled)

y_pred_STB_inv = scaler_y_STB.inverse_transform(y_pred_STB_scaled)
y_test_STB_inv = scaler_y_STB.inverse_transform(y_test_STB_scaled)

In [11]:
# Métricas reales
print("Flow Rate Prop. BR:")
print(f"MAE: {mean_absolute_error(y_test_BR_inv, y_pred_BR_inv):.2f}")
print(f"MSE: {mean_squared_error(y_test_BR_inv, y_pred_BR_inv):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test_BR_inv, y_pred_BR_inv)):.2f}")
print(f"R²: {r2_score(y_test_BR_inv, y_pred_BR_inv):.4f}\n")

print("Flow Rate Prop. STB:")
print(f"MAE: {mean_absolute_error(y_test_STB_inv, y_pred_STB_inv):.2f}")
print(f"MSE: {mean_squared_error(y_test_STB_inv, y_pred_STB_inv):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test_STB_inv, y_pred_STB_inv)):.2f}")
print(f"R²: {r2_score(y_test_STB_inv, y_pred_STB_inv):.4f}")

Flow Rate Prop. BR:
MAE: 2.26
MSE: 40.19
RMSE: 6.34
R²: 0.9861

Flow Rate Prop. STB:
MAE: 2.81
MSE: 88.79
RMSE: 9.42
R²: 0.9648
